# BDP Multilingual Bot

---

Based on multilingual bot model, i will try to train it in QA with English and Spanish QA 

The dataset that i would use for train and test this case will be QA in English and Spanish I have combined for this model TriviaQA dataset and SQuAD_es dataset from this sites: 
 - https://github.com/ccasimiro88/TranslateAlignRetrieve
 - http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz

 This model, as part of BERT will be espcialist in answer questiosn, but needing a context, or a multiple choice

## Prepare Dataset

First I am going to create and use the information that i will use from TriviaQA dataset, selecting the params:
- Question
- Answer
- Search Results that are the context 

In [1]:
import requests
import tarfile
import os
url = "http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz"
target_path = 'triviaqa-unfiltered.tar.gz'

#Download the file if it doesn't exist
if target_path not in os.listdir():
    print("Downloading file")
    # Dowload the file 
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        print("Saving file")
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

    # Unzip the file
    with tarfile.open(target_path, "r:gz") as tar:
        tar.extractall(path=".")
else:
    print("File already exists")

File already exists


In [2]:
# create a  Train Dataframe from the json file with the variables that i need of TriviaQA
import os
import pandas as pd
import json

if 'triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-train.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = "Example context:"
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'context': context,
                'question': question,
                'answers': [answer]
            })
        # Create a DataFrame from the list of dictionaries
        df_tr = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        df_tr.to_csv('triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame
    else:
        print("Json has not the dxpected format")
else:
    df_tr = pd.read_csv('triviaqa-unfiltered.csv')
    
df_tr.head()
df_tr.dropna(inplace=True)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# create a  Test Dataframe from the json file with the variables that i need of TriviaQA

if 'test-triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-dev.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'context': context,
                'question': question,
                'answers': [answer]
            })

        # Create a DataFrame from the list of dictionaries
        test_tr = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        test_tr.to_csv('test-triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame

    else:
        print("Json has not the dxpected format")
else:
    test_tr = pd.read_csv('test-triviaqa-unfiltered.csv')
test_tr.dropna(inplace=True)
test_tr.head()

Now i am going to prepare the SQuAD-es dataset, selecting:
- Context
- Questions
- Answers 

In [ ]:
# Download dataset if it doesn't exist
import os
from datasets import load_dataset

target_path = 'squad_es'

if target_path not in os.listdir():
        # Dowload the file showing the progress bar
    print("Downloading file")
    dataset = load_dataset("squad_es", 'v1.1.0')  
    dataset.save_to_disk(target_path)  
else:
    print("File already exists")

In [ ]:
from datasets import load_from_disk
import pandas as pd

# Cargar el conjunto de datos desde el disco
if "squad_train.csv" not in os.listdir() or "squad_test.csv" not in os.listdir():
    dataset = load_from_disk('squad_es')

    # Convertir el conjunto de datos en DataFrame de Pandas
    # Asumiendo que quieres convertir la parte 'train' del conjunto de datos
    df_sq = pd.DataFrame(dataset['train'])
    test_sq = pd.DataFrame(dataset['validation'])

    # eliminate the columns that i dont need
    df_sq.pop('id')
    df_sq.pop('title')
    test_sq.pop('id')
    test_sq.pop('title')  
    
    df_sq['answers'] = df_sq['answers'].apply(lambda x: x['text'])
    test_sq['answers'] = test_sq['answers'].apply(lambda x: x['text'])


    df_sq.to_csv('squad_train.csv', index=False)
    test_sq.to_csv('squad_test.csv', index=False)
    

else:
    df_sq = pd.read_csv('squad_train.csv')
    test_sq = pd.read_csv('squad_test.csv')
    


In [ ]:
# Now, combine the two dataframes in one, mix the dataframes

if 'train.csv' not in os.listdir() or 'test.csv' not in os.listdir():
    df = pd.concat([df_tr, df_sq], ignore_index=True)
    test = pd.concat([test_tr, test_sq], ignore_index=True)
else:
    df = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')


In [ ]:
df.dropna(inplace=True)
test.dropna(inplace=True)

Now, we have got both dataframes combined, lets tokenize the information

In [ ]:

from transformers import BertTokenizer
import torch

# Verify if Cuda is available to use GPU
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

# use multilingual cased model and its tokenizer
tokenizer_cased = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# fucntion to get the start and end indexes 
def find_start_end_indices(row):
    # Tokenize the context
    context_tokens = tokenizer_cased.encode(row['context'], add_special_tokens=False)
    answer_tokens = tokenizer_cased.encode(row['answer'], add_special_tokens=False)

    # search start an end context
    start_index = None
    end_index = None
    for i in range(len(context_tokens)):
        if context_tokens[i:i+len(answer_tokens)] == answer_tokens:
            start_index = i
            end_index = i + len(answer_tokens) - 1
            break

    return start_index, end_index

In [ ]:


if not os.path.exists('train.csv'):
    # Function to tokenize the text
    print("Tokenizing the text")
    def tokenize(text):
        return tokenizer_cased.encode_plus(
            text,
            add_special_tokens=True,#add [CLS], [SEP] (special tokens) at the beginning and end of the sentence
            max_length=512, #maximum length of a sentence in tokens
            padding='max_length', #add [PAD]s at the end of sentences to reach max_length
            truncation=True, #truncates sentences to max_length if they exceed it
            return_attention_mask=True, #return attention masks to know what tokens to attend to
            return_tensors='pt' #return PyTorch tensors (also works with TensorFlow) being able to use them directly in PyTorch models with GPU
        )

    # Apply the tokenize function to the question and context columns
    # input_ids are the tokenized text (number that represent the word in the vocabulary)
    df['input_ids'] = df['question'].apply(lambda x: tokenize(x)['input_ids'])
    df['attention_mask'] = df['question'].apply(lambda x: tokenize(x)['attention_mask'])
    
    # obtain the start and en indexes in order to give the clue tht must search in the context
    df[['start_index', 'end_index']] = df.apply(lambda row: find_start_end_indices(row), axis=1, result_type='expand')
    test[['start_index', 'end_index']] = df.apply(lambda row: find_start_end_indices(row), axis=1, result_type='expand')

    test['input_ids'] = test['question'].apply(lambda x: tokenize(x)['input_ids'])
    test['attention_mask'] = test['question'].apply(lambda x: tokenize(x)['attention_mask'])
    df['question'] = df['question'].apply(lambda x: tokenize(x)['input_ids']) #apply the tokenize function to the question column using the input_ids
    test['question'] = test['question'].apply(lambda x: tokenize(x)['input_ids'])
    df['context'] = df['context'].apply(lambda x: tokenize(x)['input_ids'])
    test['context'] = test['context'].apply(lambda x: tokenize(x)['input_ids'])

    # due to i want a moder able to genereate a answer, i need to tokenize the answers
    # in the other hand if i want to use a model to select an answer i dont need to tokenize the answers but i could do it anyways 
    df['answers'] = df['answers'].apply(lambda x: tokenize(x)['input_ids'])
    test['answers'] = test['answers'].apply(lambda x: tokenize(x)['input_ids'])


    df.to_pickle('train.pkl')
    test.to_pickle('test.pkl')



else:
    print("Loading the tokenized text")
    df = pd.read_pickle('train.pkl')
    test = pd.read_pickle('test.pkl')

df.head()

After tokenize and convert them into tensors, it is time to create the DataLoader

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

# Split the data into train, validation and test sets
train_df, val_df = train_test_split(df, test_size=0.2)

import torch
# Suponiendo que df es tu DataFrame y que las columnas 'input_ids' y 'attention_mask' contienen tensores de PyTorch
train_dataset = TensorDataset(torch.stack(train_df['input_ids'].tolist()), torch.stack(train_df['attention_mask'].tolist()))
test_dataset = TensorDataset(torch.stack(test['input_ids'].tolist()), torch.stack(test['attention_mask'].tolist()))
val_dataset = TensorDataset(torch.stack(val_df['input_ids'].tolist()), torch.stack(val_df['attention_mask'].tolist()))

# Create Data Loader
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
from transformers import BertModel, BertForQuestionAnswering, BertForMultipleChoice, AdamW
# load this 3 models in order to compare the results of them for this case of use.

model_cased = BertModel.from_pretrained('bert-base-multilingual-cased')
qa_model = BertForQuestionAnswering.from_pretrained('bert-base-multilingual-cased')
mulchoice_model = BertForMultipleChoice.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# create a generic function to pass the model that i want to train and test, also load the model from a checkpoint saved if exists 

def train_test_model(model, name):
    # use cuda if available in order to train it faster
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # uses Adam, in the future change to tests other models
    optimizer = AdamW(model.parameters(), lr=5e-5)
    model.train()
    for batch in train_loader:
        